In [1]:
import os
os.chdir("/home/elicer/ptta") # os.chdir("/home/ubuntu/test-time-adapters")

In [2]:
# Set CUDA Device Number
DEVICE_NUM = 0

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = str(DEVICE_NUM)

In [3]:
import sys
from os import path, environ
from argparse import ArgumentParser

import torch

from ttadapters import datasets, models
from ttadapters.models.base import ModelProvider
from ttadapters.utils import visualizer, validator
from ttadapters.datasets import DatasetHolder, scenarios

from ttadapters.methods.other_method.baseline import ActMADConfig, ActMAD

/home/elicer/ptta/.venv/lib/python3.10/site-packages/detectron2/model_zoo/model_zoo.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
environ["TORCHDYNAMO_CAPTURE_SCALAR_OUTPUTS"] = "1"
environ["TORCHDYNAMO_CAPTURE_DYNAMIC_OUTPUT_SHAPE_OPS"] = "1"

torch._dynamo.config.capture_scalar_outputs = True
torch._dynamo.config.suppress_errors = True

In [5]:
# Set Batch Size
BATCH_SIZE = 2, 8, 1  # Local
#BATCH_SIZE = 40, 200, 1  # A100 or H100
ACCUMULATE_STEPS = 1

# Set Data Root
DATA_ROOT = path.join(".", "data")

# Set Target Dataset
SOURCE_DOMAIN = datasets.SHIFTDataset

# Set Run Mode
TEST_MODE = True

# Set Model List
MODEL_ZOO = ["rcnn", "swinrcnn", "yolo11", "rtdetr"]
MODEL_TYPE = MODEL_ZOO[0]

In [6]:
# Set CUDA Device Number
DEVICE_NUM = 0
ADDITIONAL_GPU = 0
DATA_TYPE = torch.float32

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))
print(f"INFO: Using data precision - {DATA_TYPE}")

INFO: Using device - cuda:0
INFO: Using data precision - torch.float32


In [7]:
# Fast download patch
datasets.patch_fast_download_for_object_detection()

In [8]:
# Basic pre-training dataset
match SOURCE_DOMAIN:
    case datasets.SHIFTDataset:
        dataset = DatasetHolder(
            train=datasets.SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, train=True),
            valid=datasets.SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, valid=True),
            test=datasets.SHIFTCorruptedDatasetForObjectDetection(root=DATA_ROOT, valid=True)
        )
    case datasets.CityscapesDataset:
        pass
    case _:
        raise ValueError(f"Unsupported dataset: {SOURCE_DOMAIN}")

# Dataset info
CLASSES = dataset.train.classes
NUM_CLASSES = len(CLASSES)
print(f"INFO: Number of classes - {NUM_CLASSES} {CLASSES}")

[10/12/2025 13:57:16] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/train. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7fc3f4dcee30>
[10/12/2025 13:57:16] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/train/front/det_2d.json' ...


INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[10/12/2025 13:57:17] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/train/front/det_2d.json' Done.
[10/12/2025 13:57:22] SHIFT DevKit - INFO - Loading annotation takes 5.55 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['0016-1b62']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                    -7.53     219.91
boxes2d              torch.Size([1, 26, 4])                    5.00     974.00
boxes2d_classes      torch.Size([1, 26])                       0.00       3.00
boxes2d_track_ids    torch.Size([1, 26])                       0.00      25.00
images               torch.Size([1, 3, 800, 1280])             0.00     255.00



[10/12/2025 13:57:24] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7fc3f4dcee30>
[10/12/2025 13:57:24] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/val/front/det_2d.json' ...
[10/12/2025 13:57:24] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/val/front/det_2d.json' Done.


Video name: 0016-1b62
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[10/12/2025 13:57:25] SHIFT DevKit - INFO - Loading annotation takes 0.51 seconds.
[10/12/2025 13:57:25] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7fc3f4dcee30>
[10/12/2025 13:57:25] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/corrupted/discrete/images/val/front/det_2d.json' ...


INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[10/12/2025 13:57:26] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/corrupted/discrete/images/val/front/det_2d.json' Done.
[10/12/2025 13:57:30] SHIFT DevKit - INFO - Loading annotation takes 5.63 seconds.


INFO: Dataset loaded successfully. Number of samples - Train: 20800, Valid: 2800, Test: 22200

INFO: Number of classes - 6 ['pedestrian', 'car', 'truck', 'bus', 'motorcycle', 'bicycle']


In [9]:
# Initialize model
match MODEL_TYPE:
    case "rcnn":
        model = models.FasterRCNNForObjectDetection(dataset=SOURCE_DOMAIN)
        load_result = model.load_from(**vars(model.Weights.SHIFT_CLEAR_NATUREYOO if TEST_MODE else model.Weights.IMAGENET_OFFICIAL), strict=False)
    case "swinrcnn":
        model = models.SwinRCNNForObjectDetection(dataset=SOURCE_DOMAIN)
        load_result = model.load_from(**vars(model.Weights.SHIFT_CLEAR_NATUREYOO if TEST_MODE else model.Weights.IMAGENET_XIAOHU2015), strict=False)
    case "rtdetr":
        model = models.RTDetrForObjectDetection(dataset=SOURCE_DOMAIN)
        load_result = model.load_from(**vars(model.Weights.SHIFT_CLEAR if TEST_MODE else model.Weights.COCO_OFFICIAL), strict=False)
    case _:
        raise ValueError(f"Unsupported model type: {MODEL_TYPE}")

print("INFO: Model state loaded -", load_result)
model.to(device)

INFO: Model state loaded - <All keys matched successfully>


FasterRCNNForObjectDetection(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Sequential(
        (0): Bo

In [10]:
config = ActMADConfig()
config.data_root = DATA_ROOT
config.clear_dataset = model.DataPreparation(dataset.train, evaluation_mode=True)
adaptive_model = ActMAD(model, config)

Extract statistic:   0%|          | 0/1300 [00:00<?, ?it/s]

In [11]:
# Compile model
adaptive_model = torch.compile(adaptive_model)

In [12]:
# Ensure split (required due to Scenario class works with coroutines)
_ = datasets.SHIFTContinuousSubsetForObjectDetection(root=DATA_ROOT, train=True)

INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/continuous/1x...
Welcome to use SHIFT Dataset download script (Multi-threaded)! 
By continuing you confirm that you have agreed to the SHIFT's user license.



[10/12/2025 14:08:19] SHIFT Downloader - INFO - Number of files to download: 10
[2025-10-12 14:08:19,054 download.py:260 main] Number of files to download: 10
[10/12/2025 14:08:19] SHIFT Downloader - INFO - Using 8 concurrent threads
[2025-10-12 14:08:19,054 download.py:261 main] Using 8 concurrent threads
[10/12/2025 14:08:19] SHIFT Downloader - INFO - Starting download: https://dl.cv.ethz.ch/shift/continuous/images/1x/train/front/img.zip
[2025-10-12 14:08:19,055 download.py:158 download_file] Starting download: https://dl.cv.ethz.ch/shift/continuous/images/1x/train/front/img.zip
[10/12/2025 14:08:19] SHIFT Downloader - INFO - Downloading - Shift: continuous/1x, Framerate: images (1 fps), Split: training set, View: Front, Data group: RGB Image.
[2025-10-12 14:08:19,055 download.py:160 download_file] Downloading - Shift: continuous/1x, Framerate: images (1 fps), Split: training set, View: Front, Data group: RGB Image.
[10/12/2025 14:08:19] SHIFT Downloader - INFO - Starting download: h

INFO: Dataset archive downloaded and extracted.
INFO: Subset split for 'SHIFT_SUBSET' dataset is started...
INFO: Processing for 1x
INFO: Splitting train
INFO: <daytime_to_night> datasets - Clear: 960, Rainy: 560, Cloudy: 720, Foggy: 480, Overcast: 200, Total: 2920/9040
INFO: <clear_to_foggy> datasets - Daytime: 1800, Night: 520, Dawn: 560, Total: 2880/9040
INFO: <clear_to_rainy> datasets - Daytime: 1600, Night: 1000, Dawn: 640, Total: 3240/9040
INFO: Splitting val
INFO: <daytime_to_night> datasets - Clear: 160, Rainy: 280, Cloudy: 200, Foggy: 80, Overcast: 160, Total: 880/2280
INFO: <clear_to_foggy> datasets - Daytime: 240, Night: 200, Dawn: 240, Total: 680/2280
INFO: <clear_to_rainy> datasets - Daytime: 360, Night: 240, Dawn: 120, Total: 720/2280
INFO: Processing for 10x
INFO: Splitting train


FileNotFoundError: [Errno 2] No such file or directory: './data/SHIFT/continuous/images/10x/train/front/det_2d.json'

In [ ]:
data_preparation = model.DataPreparation(datasets.base.BaseDataset(), evaluation_mode=True)

discrete_scenario = scenarios.SHIFTDiscreteScenario(
    root=DATA_ROOT, valid=True, order=scenarios.SHIFTDiscreteScenario.WHWPAPER, transforms=data_preparation.transforms
)
continuous_scenario = scenarios.SHIFTContinuousScenario(
    root=DATA_ROOT, valid=True, order=scenarios.SHIFTContinuousScenario.DEFAULT, transforms=data_preparation.transforms
)

In [ ]:
evaluator = validator.DetectionEvaluator(adaptive_model, classes=CLASSES, data_preparation=data_preparation, dtype=DATA_TYPE, device=device)
evaluator_loader_params = dict(batch_size=BATCH_SIZE[2], shuffle=False, collate_fn=data_preparation.collate_fn)

In [ ]:
visualizer.visualize_metrics(discrete_scenario(**evaluator_loader_params).play(evaluator, index=[adaptive_model.__class__.__name__]))

In [ ]:
visualizer.visualize_metrics(continuous_scenario(**evaluator_loader_params).play(evaluator, index=[adaptive_model.__class__.__name__]))